# Audio Embedders

In [5]:
#| default_exp audio.embedding

In [6]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#| export
from encodec import EncodecModel

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()